In [2]:
import torch

# Zielwert
target = torch.tensor(5.0)

# Parameter a, mit Gradienten
a = torch.nn.Parameter(torch.tensor(0.0))

# Optimizer
optimizer = torch.optim.SGD([a], lr=0.01)

# Fixer Vektor
v2 = torch.tensor([2.0, 1.0, 1.0])

for epoch in range(100):
    optimizer.zero_grad()

    # Vektor v1 mit Parameter a
    v1 = torch.stack([a, torch.tensor(1.0), torch.tensor(1.0)])

    # Kreuzprodukt
    cross = torch.cross(v1, v2)

    # Norm (Betrag) des Kreuzprodukts
    norm = torch.norm(cross)

    # Loss: quadratischer Abstand zur Zielgröße
    loss = (norm - target)**2

    # Rückwärts und Optimieren
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}: a = {a.item():.4f}, norm = {norm.item():.4f}, loss = {loss.item():.6f}")

print(f"\nFinaler Wert von a: {a.item():.4f}")


Epoch 0: a = -0.0614, norm = 2.8284, loss = 4.715729
Epoch 10: a = -0.5555, norm = 3.5563, loss = 2.084363
Epoch 20: a = -0.8840, norm = 4.0402, loss = 0.921293
Epoch 30: a = -1.1024, norm = 4.3619, loss = 0.407214
Epoch 40: a = -1.2475, norm = 4.5757, loss = 0.179989
Epoch 50: a = -1.3441, norm = 4.7179, loss = 0.079556
Epoch 60: a = -1.4082, norm = 4.8125, loss = 0.035164
Epoch 70: a = -1.4509, norm = 4.8753, loss = 0.015543
Epoch 80: a = -1.4793, norm = 4.9171, loss = 0.006870
Epoch 90: a = -1.4981, norm = 4.9449, loss = 0.003036

Finaler Wert von a: -1.5096
